### Ingestion del archivo "person.json"

In [0]:
dbutils.widgets.text("p_environment","")
v_environment = dbutils.widgets.get("p_environment")

In [0]:
dbutils.widgets.text("p_file_date","2024-12-30")
v_file_date = dbutils.widgets.get("p_file_date")

In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/commom_functions"

### Paso 1 - Leer el archivo JSON usando "DataFrameReader" de Spark

In [0]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

In [0]:
name_schema = StructType(fields=[
    StructField("forename", StringType(), True),
    StructField("surname", StringType(), True)
])

In [0]:
person_schema = StructType(fields=[
    StructField("personId", IntegerType(), False),
    StructField("personName", name_schema)
])

In [0]:
person_df = spark.read \
            .schema(person_schema) \
            .json(f"{bronze_folder_path}/{v_file_date}/person.json")

### Paso 2 - Renombar las columnas y añadir nuevas columnas
1. "personid" renombrar a "person_id"
2. Agregar las columnas "ingestion_date" y "environment"
3. Agregar la columna "name" a partir de la concatenacion de "forname" y "surname"

In [0]:
from pyspark.sql.functions import col,concat, current_timestamp, lit

In [0]:
persons_with_columns_df = add_ingestion_date(person_df) \
                         .withColumn("environment", lit(v_environment)) \
                         .withColumn("file_date",lit(v_file_date)) \
                         .withColumnRenamed("personId", "person_id") \
                         .withColumn("name",
                                     concat(
                                            col("personName.forename"),
                                            lit(" "), 
                                            col("personName.surname")
                                            )
                                     )

### Paso 3 - Eliminar las columnas no "requeridas"

In [0]:
persons_final_df = persons_with_columns_df.drop(col("personName"))

In [0]:
display(persons_final_df)

### Paso 4 - Escribir la salida en un formato "Parquet"

In [0]:
#overwrite_partition(persons_final_df,"movie_silver","persons","file_date")

In [0]:
# persons_final_df.write.mode("overwrite").parquet(f"{silver_folder_path}/persons")

# Crear una tabla en base al data frame persons_final_df
# Creara la tabla persons en a DB que se llame movie_silver
# Modulo: Spark SQL - Database / Table / View: seccion: Crear Tabla - Capa Silver
#persons_final_df.write.mode("overwrite").format("parquet").saveAsTable("movie_silver.persons")

#Nota: Esto es una tabla administrada por Databricks, esto quiere decir que si la tabla se elimina, los registros tambien se van a eliminar
# Append, agrega los registros a la tabla existente, carga incremental de datos

#persons_final_df.write.mode("append").partitionBy("file_date").format("parquet").saveAsTable("movie_silver.persons")

#Modulo: Delta Lake
merge_condition = "tgt.person_id = src.person_id AND tgt.file_date = src.file_date" # Buscar por particion
merge_delta_lake(persons_final_df,"movie_silver","persons",silver_folder_path,merge_condition,"file_date")

In [0]:
%sql
--Consultar la BD movie_silver, a tabla persons
-- Tabla Administrada por Databricks
SELECT file_date, COUNT(1)
FROM movie_silver.persons
GROUP BY 1;

In [0]:
#display(spark.read.parquet(f"{silver_folder_path}/persons"))

In [0]:
dbutils.notebook.exit("Success")